In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
imageSize = (224, 224)
vgg = VGG16(input_shape=imageSize + (3,), weights='imagenet', include_top=False)

In [3]:
for layer in vgg.layers:
    layer.trainable=False

In [4]:
x=Flatten()(vgg.output)

In [28]:
x=Dense(128,activation='relu')(x)

In [5]:
output_layer=Dense(8,activation='softmax')(x)

In [6]:
model=Model(inputs=vgg.input,outputs=output_layer)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
model.compile(
   loss='categorical_crossentropy',
   optimizer='adam',
   metrics=['accuracy'], run_eagerly=True
)

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_data_dir = 'training dataset'
test_data_dir = 'testing dataset'

batch_size = 10
target_size = (224, 224) 

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'  # Set to 'int' or 'categorical' based on your label encoding
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'  # Set to 'int' or 'categorical' based on your label encoding
)

# Print the list of classes
class_labels = train_generator.class_indices
print("List of classes:", class_labels)

Found 706 images belonging to 8 classes.
Found 179 images belonging to 8 classes.
List of classes: {'Anthracnose': 0, 'algal leaf': 1, 'bird eye spot': 2, 'brown blight': 3, 'gray light': 4, 'healthy': 5, 'red leaf spot': 6, 'white spot': 7}


In [ ]:
# Define the number of epochs
num_epochs = 30

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

# Calculate and print the final accuracy
final_accuracy = history.history['val_accuracy'][-1]
print("Accuracy after 30 epochs:", final_accuracy)


C:\Users\91999\AppData\Local\Temp\ipykernel_12280\1636968778.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
71/71 [==============================] - 396s 6s/step - loss: 1.9508 - accuracy: 0.4377 - val_loss: 0.9071 - val_accuracy: 0.6201
Epoch 2/30
71/71 [==============================] - 409s 6s/step - loss: 0.8063 - accuracy: 0.7040 - val_loss: 0.8536 - val_accuracy: 0.6760
Epoch 3/30
71/71 [==============================] - 404s 6s/step - loss: 0.6805 - accuracy: 0.7606 - val_loss: 0.7896 - val_accuracy: 0.7207
Epoch 4/30
71/71 [==============================] - 406s 6s/step - loss: 0.4716 - accuracy: 0.8329 - val_loss: 0.7678 - val_accuracy: 0.7095
Epoch 5/30
71/71 [==============================] - 413s 6s/step - loss: 0.3745 - accuracy: 0.8824 - val_loss: 0.9218 - val_accuracy: 0.6816
Epoch 6/30
71/71 [==============================] - 408s 6s/step - loss: 0.2919 - accuracy: 0.9108 - val_loss: 0.7041 - val_accuracy: 0.7877
Epoch 7/30
71/71 [==============================] - 403s 6s/step - loss: 0.2241 - accuracy: 0.9249 - val_loss: 0.5996 - val_accuracy: 0.7877
Epoch 8/30
71

In [ ]:
model.save("vgg16-predicting-disease-in-tea.h5")